In [49]:
import pandas as pd
import openpyxl

In [50]:
# 使用openpyxl加载工作簿
wb = openpyxl.load_workbook('data/商机信息统计.xlsx')
sheet = wb.active

# 获取列字母（A, B, C等）
def get_column_letter(header_name):
    for cell in sheet[1]:  # 第一行
        if cell.value == header_name:
            return cell.column_letter
    return None

# 获取网站链接和原始链接的列
website_col = get_column_letter('网站链接')
original_col = get_column_letter('原始链接')

# 直接读取这两列的所有单元格
if website_col:
    website_cells = sheet[website_col][1:]  # 跳过标题行
if original_col:
    original_cells = sheet[original_col][1:]  # 跳过标题行

# 获取链接
website_links = []
original_links = []

if website_col:
    for cell in website_cells:
        if cell.hyperlink:
            website_links.append(cell.hyperlink.target)
        else:
            website_links.append(cell.value)  # 如果不是超链接，保留原始值

if original_col:
    for cell in original_cells:
        if cell.hyperlink:
            original_links.append(cell.hyperlink.target)
        else:
            original_links.append(cell.value)  # 如果不是超链接，保留原始值

# 使用pandas读取Excel
data = pd.read_excel('data/商机信息统计.xlsx', engine='openpyxl')

# 更新链接列
if website_col:
    data['网站链接'] = website_links
if original_col:
    data['原始链接'] = original_links



In [51]:
import re
import unicodedata

#判断单元格是否为中文的函数
def is_chinese(char):
    code_point = ord(char)
    return 0x4E00 <= code_point <= 0x9FFF

def non_chinese_ratio(text):
    if not text or pd.isna(text): 
        return 0.0
    total_chars = len(text)
    non_chinese_chars = sum(1 for char in text if not is_chinese(char))
    ratio=(non_chinese_chars/total_chars)*100
    return ratio

data["乱码"]=data["公告正文"].apply(non_chinese_ratio)>=70.0
data.head(5)


,类型,标题,项目标签,业务相关性,公告摘要,公告正文,预算金额(万),网站链接,原始链接,采购单位,招标代理,联系方式,发布日期,报名截至日期,预计采购日期,销售部门,客户代表,信息采集时间,处理标志,乱码
0,招标,中国福利彩票发行管理中心2025年信息监测及相关服务项目公开招标公告,北京市-朝阳区|民政,一般,【项目概况】： 2025年信息监测及相关服务项目，预算金额：152.42万元人民币。\n【项...,公告正文\n项目概况\n2025年信息监测及相关服务项目 招标项目的潜在投标人应在中国机械进...,152.42,"=HYPERLINK(""https://www.jianyu360.cn/page_work...","=HYPERLINK(""https://www.ccgp.gov.cn/cggg/zygg/...",中国福利彩票发行管理中心,中国机械进出口集团有限公司,【采购方联系方式】：\n李先生，010-58125705\n【采购方联系方式】：\n陈女士，...,2025-04-29,2025-05-23,NaN,行业推进事业部,陈宣霖,2025-04-30 11:28:51.159,NaN,False
1,招标,中国福利彩票发行管理中心2025年走近双色球项目公开招标公告,北京市-朝阳区|民政,无关,【项目概况】： 中国福利彩票发行管理中心2025年走近双色球项目，预算金额329.52万元。...,公告正文\n项目概况\n中国福利彩票发行管理中心2025年走近双色球项目 招标项目的潜在投标...,329.52,"=HYPERLINK(""https://www.jianyu360.cn/page_work...","=HYPERLINK(""https://www.ccgp.gov.cn/cggg/zygg/...",中国福利彩票发行管理中心,中技国际招标有限公司,【采购方联系方式】：\n李先生，010-58125705\n【采购方联系方式】：\n洪林杨、...,2025-04-29,2025-05-09,NaN,行业推进事业部,陈宣霖,2025-04-30 11:28:19.904,NaN,False
2,竞谈,嘉定校区体育中心泳池池底维修采购项目,上海市-杨浦区|学校,无关,【项目概况】： 工程建设，预算金额：97.788万元\n【项目内容】： 体育中心泳池池底瓷砖...,公告正文\n磋商公告\n一、项目概况\n项目名称：嘉定校区体育中心泳池池底维修\n项目编号：...,97.78,"=HYPERLINK(""https://www.jianyu360.cn/page_work...","=HYPERLINK(""https://czb.tongji.edu.cn/sggl/cgw...",同济大学,上海华建工程建设咨询有限公司,【采购方联系方式】：\n肖老师，021-65983577\n【采购方联系方式】：\n唐寅，0...,2025-04-29,2025-05-03,NaN,教育事业一部,张潞,2025-04-30 11:27:30.202,NaN,False
3,招标,海洋科学传感器一批采购项目招标公告,上海市-杨浦区|学校,无关,【项目概况】： 硬件采购，预算金额：人民币112.3万元\n【项目内容】： 采购4套声学多普...,公告正文\n海洋科学传感器一批采购（重）-国际招标公告\n上海中世建设咨询有限公司受招标人委...,112.30,"=HYPERLINK(""https://www.jianyu360.cn/page_work...","=HYPERLINK(""https://czb.tongji.edu.cn/sggl/cgw...",同济大学,上海中世建设咨询有限公司,【采购方联系方式】：\n徐老师，021-20926959-1046\n【采购方联系方式】：\...,2025-04-29,2025-05-09,NaN,教育事业一部,张潞,2025-04-30 11:26:59.563,NaN,False
4,招标,同济大学海洋科学传感器一批采购（重）-国际招标公告,上海市-杨浦区|学校,无关,【项目概况】： 硬件采购，项目预算112.3万元。\n【项目内容】： 采购4套声学多普勒流速...,公告正文\n上海中世建设咨询有限公司受同济大学 委托，根据《中华人民共和国政府采购法》等有关...,112.30,"=HYPERLINK(""https://www.jianyu360.cn/page_work...","=HYPERLINK(""https://www.ccgp.gov.cn/cggg/zygg/...",同济大学,上海中世建设咨询有限公司,【采购方联系方式】：\n徐老师，021-20926959-1046\n【采购方联系方式】：\...,2025-04-29,2025-05-09,NaN,教育事业一部,张潞,2025-04-30 11:26:29.846,NaN,False


In [52]:
data=data[data["乱码"]==False]
data=data[["类型","标题","项目标签","业务相关性","公告摘要","公告正文","采购单位","招标代理"]]
data

,类型,标题,项目标签,业务相关性,公告摘要,公告正文,采购单位,招标代理
0,招标,中国福利彩票发行管理中心2025年信息监测及相关服务项目公开招标公告,北京市-朝阳区|民政,一般,【项目概况】： 2025年信息监测及相关服务项目，预算金额：152.42万元人民币。\n【项...,公告正文\n项目概况\n2025年信息监测及相关服务项目 招标项目的潜在投标人应在中国机械进...,中国福利彩票发行管理中心,中国机械进出口集团有限公司
1,招标,中国福利彩票发行管理中心2025年走近双色球项目公开招标公告,北京市-朝阳区|民政,无关,【项目概况】： 中国福利彩票发行管理中心2025年走近双色球项目，预算金额329.52万元。...,公告正文\n项目概况\n中国福利彩票发行管理中心2025年走近双色球项目 招标项目的潜在投标...,中国福利彩票发行管理中心,中技国际招标有限公司
2,竞谈,嘉定校区体育中心泳池池底维修采购项目,上海市-杨浦区|学校,无关,【项目概况】： 工程建设，预算金额：97.788万元\n【项目内容】： 体育中心泳池池底瓷砖...,公告正文\n磋商公告\n一、项目概况\n项目名称：嘉定校区体育中心泳池池底维修\n项目编号：...,同济大学,上海华建工程建设咨询有限公司
3,招标,海洋科学传感器一批采购项目招标公告,上海市-杨浦区|学校,无关,【项目概况】： 硬件采购，预算金额：人民币112.3万元\n【项目内容】： 采购4套声学多普...,公告正文\n海洋科学传感器一批采购（重）-国际招标公告\n上海中世建设咨询有限公司受招标人委...,同济大学,上海中世建设咨询有限公司
4,招标,同济大学海洋科学传感器一批采购（重）-国际招标公告,上海市-杨浦区|学校,无关,【项目概况】： 硬件采购，项目预算112.3万元。\n【项目内容】： 采购4套声学多普勒流速...,公告正文\n上海中世建设咨询有限公司受同济大学 委托，根据《中华人民共和国政府采购法》等有关...,同济大学,上海中世建设咨询有限公司
...,...,...,...,...,...,...,...,...
5626,招标,上海中医药大学附属曙光医院学生宿舍租赁项目公开招标公告,上海市-浦东新区|医疗,无关,【项目概况】： 学生宿舍租赁项目，预算为二室一厅5400元/套/月，三室一厅7200元/套/...,公告正文\n \n上海中医药大学附属曙光医院学生宿舍租赁项目公开招标公告\n(招标编号:XS...,上海中医药大学附属曙光医院,上海欣声招标服务中心有限公司
5627,竞谈,上海中医药大学附属曙光医院宾得电子支气管镜及主机维保项目,上海市-长宁区|医疗,无关,【项目概况】： 医疗器械维保项目，项目预算未明确。\n【项目内容】： 宾得电子支气管镜及主机...,公告正文\n上海中医药大学附属曙光医院宾得电子支气管镜及主机维保项目（招标编号：/）\n项目...,上海中医药大学附属曙光医院,上海财瑞建设管理有限公司
5628,变更,上海中医药大学附属岳阳中西医结合医院医责险招标更正公告,上海市|医疗,无关,【项目概况】： 医责险招标，项目预算未提及。\n【项目内容】： 医责险采购，具体数量及内容未...,公告正文\n上海中医药大学附属岳阳中西医结合医院医责险招标更正公告（招标编号：招 2024-...,上海中医药大学附属岳阳中西医结合医院,上海财瑞建设管理有限公司
5629,招标,同济大学四平路校区综合运动馆项目勘察设计采购项目,上海市|学校,潜在商机,【项目概况】： 工程建设，项目预算14482万元。\n【项目内容】： 拟建一幢综合运动馆，总...,公告正文\n招标公告\n公开招标信息表\n报建编号：\n2501YP0012 \n标段号：\...,同济大学,上海教育建设管理咨询有限公司


In [53]:
单位词典=set()
for i in data["采购单位"]:
    if i not in 单位词典:
        单位词典.add(i)
print(单位词典)
with open("dict.txt","w",encoding="utf-8") as f:
    for i in 单位词典:
        f.write(i + "\n")


{'上海大学文学院', '上海市养志康复医院', '上海交通大学医学院附属仁济医院安徽医院', '上海立信会计金融学院后勤保障处', '上海市儿童医院', '上海市皮肤病医院', '上海体育大学、安徽省南苑建设工程有限公司', '上海理工大学', '复旦大学附属中山医院国家医学中心', '上海市皮肤病医院后勤保障部采购办公室', '上海立信会计金融学院', '上海师范大学附属贵安新区实验学校', '上海市疾病预防控制中心', '上海理工大学附属实验初级中学', '上海海事大学附属职业技术学校', '上海交通大学医学院附属瑞金医院卢湾分院', '上海海事大学教育发展基金会', '中国教育工会上海电子信息职业技术学院委员会', '上海交通大学医学院附属瑞金医院、上海建工四建集团有限公司', '上海电力大学工会', '上海大学新闻传播学院', '复旦大学', '上海市水产研究所', '上海城建职业学院', '上海开放大学浦东南校', '上海中医药大学、上海公路桥梁（集团）有限公司', '上海市乡村振兴研究中心', '上海师范大学附属儋州实验学校', '上海市第六人民医院工会', '上海市第六人民医院福建医院', '上海大学材料科学与工程学院', '上海市皮肤病医院后勤保障部', '上海市血液中心', '上海大学出版社有限公司', '东华大学松江校区', '上海市第六人民医院金山分院工会', '上海政法学院团委', '华东政法大学,上海建工二建集团有限公司', '上海政法学院图书馆（档案馆）', '上海市动物疫病预防控制中心', '上海市卫生健康统计中心', '上海市科技艺术教育中心', '上海大学上海电影学院', '上海电力大学', '上海市公共卫生临床中心工会委员会', '上海师范大学附属七星关区实验学校', '上海师范大学附属第五嘉定实验学校', '复旦大学附属华山医院虹桥院区', '上海民航职业技术学院', '上海外国语大学附属外国语学校', '上海市质子重离子医院有限公司', '上海交通大学医学院', '复旦大学附属妇产科医院工会委员会', '东华大学、中国二十冶集团有限公司', '上海大学后勤保障部', '上海外国语大学三亚附属中学', '上海中医药大学附属龙华医院工会委员会', '复旦大学宁波研究院', '复旦大学附属儿科医院', '上海市农业技术推广服务中心

In [54]:
import jieba
import re

# 重新加载词典前先重置jieba
jieba.initialize()  # 重置jieba
print("Loading custom dictionary...")
jieba.load_userdict("dict.txt")

def 精确分词(text):
    # 使用精确模式分词
    seg_list = jieba.cut(text, cut_all=False)
    return list(seg_list)


data['标题分词'] = data['标题'].apply(精确分词)

# 显示前5行结果
print("\n前5行分词结果：")
data[['标题', '标题分词']].head()
data

Loading custom dictionary...

前5行分词结果：


,类型,标题,项目标签,业务相关性,公告摘要,公告正文,采购单位,招标代理,标题分词
0,招标,中国福利彩票发行管理中心2025年信息监测及相关服务项目公开招标公告,北京市-朝阳区|民政,一般,【项目概况】： 2025年信息监测及相关服务项目，预算金额：152.42万元人民币。\n【项...,公告正文\n项目概况\n2025年信息监测及相关服务项目 招标项目的潜在投标人应在中国机械进...,中国福利彩票发行管理中心,中国机械进出口集团有限公司,"[中国福利彩票发行管理中心, 2025, 年, 信息监测, 及, 相关, 服务项目, 公开招..."
1,招标,中国福利彩票发行管理中心2025年走近双色球项目公开招标公告,北京市-朝阳区|民政,无关,【项目概况】： 中国福利彩票发行管理中心2025年走近双色球项目，预算金额329.52万元。...,公告正文\n项目概况\n中国福利彩票发行管理中心2025年走近双色球项目 招标项目的潜在投标...,中国福利彩票发行管理中心,中技国际招标有限公司,"[中国福利彩票发行管理中心, 2025, 年, 走近, 双色球, 项目, 公开招标, 公告]"
2,竞谈,嘉定校区体育中心泳池池底维修采购项目,上海市-杨浦区|学校,无关,【项目概况】： 工程建设，预算金额：97.788万元\n【项目内容】： 体育中心泳池池底瓷砖...,公告正文\n磋商公告\n一、项目概况\n项目名称：嘉定校区体育中心泳池池底维修\n项目编号：...,同济大学,上海华建工程建设咨询有限公司,"[嘉定, 校区, 体育中心, 泳池, 池底, 维修, 采购, 项目]"
3,招标,海洋科学传感器一批采购项目招标公告,上海市-杨浦区|学校,无关,【项目概况】： 硬件采购，预算金额：人民币112.3万元\n【项目内容】： 采购4套声学多普...,公告正文\n海洋科学传感器一批采购（重）-国际招标公告\n上海中世建设咨询有限公司受招标人委...,同济大学,上海中世建设咨询有限公司,"[海洋, 科学, 传感器, 一批, 采购, 项目, 招标, 公告]"
4,招标,同济大学海洋科学传感器一批采购（重）-国际招标公告,上海市-杨浦区|学校,无关,【项目概况】： 硬件采购，项目预算112.3万元。\n【项目内容】： 采购4套声学多普勒流速...,公告正文\n上海中世建设咨询有限公司受同济大学 委托，根据《中华人民共和国政府采购法》等有关...,同济大学,上海中世建设咨询有限公司,"[同济大学, 海洋, 科学, 传感器, 一批, 采购, （, 重, ）, -, 国际, 招标..."
...,...,...,...,...,...,...,...,...,...
5626,招标,上海中医药大学附属曙光医院学生宿舍租赁项目公开招标公告,上海市-浦东新区|医疗,无关,【项目概况】： 学生宿舍租赁项目，预算为二室一厅5400元/套/月，三室一厅7200元/套/...,公告正文\n \n上海中医药大学附属曙光医院学生宿舍租赁项目公开招标公告\n(招标编号:XS...,上海中医药大学附属曙光医院,上海欣声招标服务中心有限公司,"[上海中医药大学附属曙光医院, 学生宿舍, 租赁, 项目, 公开招标, 公告]"
5627,竞谈,上海中医药大学附属曙光医院宾得电子支气管镜及主机维保项目,上海市-长宁区|医疗,无关,【项目概况】： 医疗器械维保项目，项目预算未明确。\n【项目内容】： 宾得电子支气管镜及主机...,公告正文\n上海中医药大学附属曙光医院宾得电子支气管镜及主机维保项目（招标编号：/）\n项目...,上海中医药大学附属曙光医院,上海财瑞建设管理有限公司,"[上海中医药大学附属曙光医院, 宾得, 电子, 支气管镜, 及, 主机, 维保, 项目]"
5628,变更,上海中医药大学附属岳阳中西医结合医院医责险招标更正公告,上海市|医疗,无关,【项目概况】： 医责险招标，项目预算未提及。\n【项目内容】： 医责险采购，具体数量及内容未...,公告正文\n上海中医药大学附属岳阳中西医结合医院医责险招标更正公告（招标编号：招 2024-...,上海中医药大学附属岳阳中西医结合医院,上海财瑞建设管理有限公司,"[上海中医药大学附属岳阳中西医结合医院, 医责险, 招标, 更正, 公告]"
5629,招标,同济大学四平路校区综合运动馆项目勘察设计采购项目,上海市|学校,潜在商机,【项目概况】： 工程建设，项目预算14482万元。\n【项目内容】： 拟建一幢综合运动馆，总...,公告正文\n招标公告\n公开招标信息表\n报建编号：\n2501YP0012 \n标段号：\...,同济大学,上海教育建设管理咨询有限公司,"[同济大学, 四平路, 校区, 综合, 运动馆, 项目, 勘察, 设计, 采购, 项目]"


In [55]:
#wuwuu